---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [0]:
# import numpy as np
# import pandas as pd
# from google.colab import files
# uploaded = files.upload()
# dates.txt

In [5]:
# Load the data
# Reference: https://necromuralist.github.io/data_science/posts/extracting-dates-from-medical-data/
import pandas

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

data = pandas.Series(doc)
data.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [6]:
data.describe()

count                                          500
unique                                         500
top       craniotomy 8/1986 Prior relevant labs:\n
freq                                             1
dtype: object

In [0]:
# 4 The Grammar
# 4.1 Cardinality
ZERO_OR_MORE = '*'
ONE_OR_MORE = "+"
ZERO_OR_ONE = '?'
EXACTLY_TWO = "{2}"
ONE_OR_TWO = "{1,2}"
EXACTLY_ONE = '{1}'

In [0]:
# 4.2 Groups and Classes
GROUP = r"({})"
NAMED = r"(?P<{}>{})"
CLASS = "[{}]"
NEGATIVE_LOOKAHEAD = "(?!{})"
NEGATIVE_LOOKBEHIND = "(?<!{})"
POSITIVE_LOOKAHEAD = "(?={})"
POSITIVE_LOOKBEHIND = "(?<={})"
ESCAPE = "\{}"

In [0]:
# 4.3 Numbers
DIGIT = r"\d"
ONE_DIGIT  = DIGIT + EXACTLY_ONE
ONE_OR_TWO_DIGITS = DIGIT + ONE_OR_TWO
NON_DIGIT = NEGATIVE_LOOKAHEAD.format(DIGIT)
TWO_DIGITS = DIGIT + EXACTLY_TWO
THREE_DIGITS = DIGIT + "{3}"
EXACTLY_TWO_DIGITS = DIGIT + EXACTLY_TWO + NON_DIGIT
FOUR_DIGITS = DIGIT + r"{4}" + NON_DIGIT

In [0]:
# 4.4 String Literals
SLASH = r"/"
OR = r'|'
LOWER_CASE = "a-z"
SPACE = "\s"
DOT = "."
DASH = "-"
COMMA = ","
PUNCTUATION = CLASS.format(DOT + COMMA + DASH)
EMPTY_STRING = ""

In [0]:
# 4.5 Dates
# These are parts to build up the date-expressions.
MONTH_SUFFIX = (CLASS.format(LOWER_CASE) + ZERO_OR_MORE
                + CLASS.format(SPACE + DOT + COMMA + DASH) + ONE_OR_TWO)
MONTH_PREFIXES = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()
MONTHS = [month + MONTH_SUFFIX for month in MONTH_PREFIXES]
MONTHS = GROUP.format(OR.join(MONTHS))
DAY_SUFFIX = CLASS.format(DASH + COMMA + SPACE) + ONE_OR_TWO
DAYS = ONE_OR_TWO_DIGITS + DAY_SUFFIX
YEAR = FOUR_DIGITS

In [0]:
# This is for dates like Mar 21st, 2009, those with suffixes on the days.
CONTRACTED = (ONE_OR_TWO_DIGITS
              + LOWER_CASE
              + EXACTLY_TWO
              )
CONTRACTION = NAMED.format("contraction",
                           MONTHS
                           + CONTRACTED
                           + DAY_SUFFIX
                           + YEAR)

In [0]:
# This is for dates that have no days in them, like May 2009.
NO_DAY_BEHIND = NEGATIVE_LOOKBEHIND.format(DIGIT + SPACE)
NO_DAY = NAMED.format("no_day", NO_DAY_BEHIND + MONTHS + YEAR)

In [0]:
# This is for the most common form (that I use) - May 21, 2017.
WORDS = NAMED.format("words", MONTHS + DAYS + YEAR)

In [0]:
BACKWARDS = NAMED.format("backwards", ONE_OR_TWO_DIGITS + SPACE + MONTHS + YEAR)

In [0]:
slashed = SLASH.join([ONE_OR_TWO_DIGITS,
                      ONE_OR_TWO_DIGITS,
                      EXACTLY_TWO_DIGITS])
dashed = DASH.join([ONE_OR_TWO_DIGITS,
                    ONE_OR_TWO_DIGITS,
                    EXACTLY_TWO_DIGITS])
TWENTIETH_CENTURY = NAMED.format("twentieth",
                                 OR.join([slashed, dashed]))

In [0]:
NUMERIC = NAMED.format("numeric",
                       SLASH.join([ONE_OR_TWO_DIGITS,
                                   ONE_OR_TWO_DIGITS,
                                   FOUR_DIGITS]))

In [0]:
NO_PRECEDING_SLASH = NEGATIVE_LOOKBEHIND.format(SLASH)
NO_PRECEDING_SLASH_DIGIT = NEGATIVE_LOOKBEHIND.format(CLASS.format(SLASH + DIGIT))
NO_ONE_DAY = (NO_PRECEDING_SLASH_DIGIT
              + ONE_DIGIT
              + SLASH
              + FOUR_DIGITS)
NO_TWO_DAYS = (NO_PRECEDING_SLASH
               + TWO_DIGITS
               + SLASH
               + FOUR_DIGITS)
NO_DAY_NUMERIC = NAMED.format("no_day_numeric",
                              NO_ONE_DAY
                              + OR
                              + NO_TWO_DAYS
                              )

In [0]:
CENTURY = GROUP.format('19' + OR + "20") + TWO_DIGITS
DIGIT_SLASH = DIGIT + SLASH
DIGIT_DASH = DIGIT + DASH
DIGIT_SPACE = DIGIT + SPACE
LETTER_SPACE = CLASS.format(LOWER_CASE) + SPACE
COMMA_SPACE = COMMA + SPACE
YEAR_PREFIX = NEGATIVE_LOOKBEHIND.format(OR.join([
    DIGIT_SLASH,
    DIGIT_DASH,
    DIGIT_SPACE,
    LETTER_SPACE,
    COMMA_SPACE,
]))

YEAR_ONLY = NAMED.format("year_only",
                         YEAR_PREFIX + CENTURY
)

In [0]:
IN_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format('iI') + 'n' + SPACE) + CENTURY
SINCE_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format("Ss") + 'ince' + SPACE) + CENTURY
AGE = POSITIVE_LOOKBEHIND.format("Age" + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
AGE_COMMA = POSITIVE_LOOKBEHIND.format("Age" + COMMA + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
OTHERS = ['delivery', "quit", "attempt", "nephrectomy", THREE_DIGITS]
OTHERS = [POSITIVE_LOOKBEHIND.format(label + SPACE) + CENTURY for label in OTHERS]
OTHERS = OR.join(OTHERS)
LEFTOVERS_PREFIX = OR.join([IN_PREFIX, SINCE_PREFIX, AGE, AGE_COMMA]) + OR + OTHERS
LEFTOVERS = NAMED.format("leftovers", LEFTOVERS_PREFIX)

In [0]:
DATE = NAMED.format("date", OR.join([NUMERIC,
                                     TWENTIETH_CENTURY,
                                     WORDS,
                                     BACKWARDS,
                                     CONTRACTION,
                                     NO_DAY,
                                     NO_DAY_NUMERIC,
                                     YEAR_ONLY,
                                     LEFTOVERS]))

In [0]:
def twentieth_century(date):
    """adds a 19 to the year

    Args:
     date (re.Regex): Extracted date
    """
    month, day, year = date.group(1).split(SLASH)
    year = "19{}".format(year)
    return SLASH.join([month, day, year])

In [0]:
def take_two(line):
    match = re.search(TWENTIETH_CENTURY, line)
    if match:
        return twentieth_century(match)
    return line

In [0]:
def extract_and_count(expression, data, name):
    """extract all matches and report the count

    Args:
     expression (str): regular expression to match
     data (pandas.Series): data with dates to extratc
     name (str): name of the group for the expression

    Returns:
     tuple (pandas.Series, int): extracted dates, count
    """
    extracted = data.str.extractall(expression)[name]
    count = len(extracted)
    print("'{}' matched {} rows".format(name, count))
    return extracted, count

In [25]:
numeric, numeric_count = extract_and_count(NUMERIC, data, 'numeric')
# 'numeric' matched 25 rows
twentieth, twentieth_count = extract_and_count(TWENTIETH_CENTURY, data, 'twentieth')
# 'twentieth' matched 100 rows
words, words_count = extract_and_count(WORDS, data, 'words')
# 'words' matched 34 rows
backwards, backwards_count = extract_and_count(BACKWARDS, data, 'backwards')
# 'backwards' matched 69 rows
contraction_data, contraction = extract_and_count(CONTRACTION, data, 'contraction')
# 'contraction' matched 0 rows
no_day, no_day_count = extract_and_count(NO_DAY, data, 'no_day')
# 'no_day' matched 115 rows
no_day_numeric, no_day_numeric_count = extract_and_count(NO_DAY_NUMERIC, data,
                                                         "no_day_numeric")
# 'no_day_numeric' matched 112 rows
year_only, year_only_count = extract_and_count(YEAR_ONLY, data, "year_only")
# 'year_only' matched 15 rows
leftovers, leftovers_count = extract_and_count(LEFTOVERS, data, "leftovers")
# 'leftovers' matched 30 rows

'numeric' matched 25 rows
'twentieth' matched 100 rows
'words' matched 34 rows
'backwards' matched 69 rows
'contraction' matched 0 rows
'no_day' matched 115 rows
'no_day_numeric' matched 112 rows
'year_only' matched 15 rows
'leftovers' matched 30 rows


In [26]:
found = data.str.extractall(DATE)
total_found = len(found.date)

print("Total Found: {}".format(total_found))
print("Remaining: {}".format(len(data) - total_found))
print("Discrepancy: {}".format(total_found - (numeric_count
                                              + twentieth_count
                                              + words_count
                                              + backwards_count
                                              + contraction
                                              + no_day_count
                                              + no_day_numeric_count
                                              + year_only_count
                                              + leftovers_count)))

Total Found: 500
Remaining: 0
Discrepancy: 0


In [0]:
# Total Found: 500
# Remaining: 0
# Discrepancy: 0

In [28]:
missing = [label for label in data.index if label not in found.index.levels[0]]
try:
    print(missing[0], data.loc[missing[0]])
except IndexError:
    print("all rows matched")

all rows matched


In [0]:
# all rows matched

In [0]:
def clean(source, expression, replacement, sample=5):
    """applies the replacement to the source

    as a side-effect shows sample rows before and after

    Args:
     source (pandas.Series): source of the strings
     expression (str): regular expression to match what to replace
     replacement: function or expression to replace the matching expression
     sample (int): number of randomly chosen examples to show

    Returns:
     pandas.Series: the source with the replacement applied to it
    """
    print("Random Sample Before:")
    print(source.sample(sample))
    cleaned = source.str.replace(expression, replacement)
    print("\nRandom Sample After:")
    print(cleaned.sample(sample))
    print("\nCount of cleaned: {}".format(len(cleaned)))
    assert len(source) == len(cleaned)
    return cleaned

In [0]:
def clean_punctuation(source, sample=5):
    """removes punctuation

    Args:
     source (pandas.Series): data to clean
     sample (int): size of sample to show

    Returns:
     pandas.Series: source with punctuation removed
    """
    print("Cleaning Punctuation")
    if any(source.str.contains(PUNCTUATION)):
        source = clean(source, PUNCTUATION, EMPTY_STRING)
    return source

In [0]:
LONG_TO_SHORT = dict(January="Jan",
                     February="Feb",
                     March="Mar",
                     April="Apr",
                     May="May",
                     June="Jun",
                     July="Jul",
                     August="Aug",
                     September="Sep",
                     October="Oct",
                     November="Nov",
                     December="Dec")

# it turns out there are spelling errors in the data so this has to be fuzzy
LONG_TO_SHORT_EXPRESSION = OR.join([GROUP.format(month)
                                    + CLASS.format(LOWER_CASE)
                                    + ZERO_OR_MORE
                                    for month in LONG_TO_SHORT.values()])

def long_month_to_short(match):
    """convert long month to short

    Args:
     match (re.Match): object matching a long month

    Returns:
     str: shortened version of the month
    """
    return match.group(match.lastindex)

In [0]:
def convert_long_months_to_short(source, sample=5):
    """convert long month names to short

    Args:
     source (pandas.Series): data with months
     sample (int): size of sample to show

    Returns:
     pandas.Series: data with short months
    """
    return clean(source,
                 LONG_TO_SHORT_EXPRESSION,
                 long_month_to_short)

In [0]:
def add_month_date(match):
    """adds 01/01 to years

    Args:
     match (re.Match): object that only matched a 4-digit year

    Returns:
     str: 01/01/YYYY
    """
    return "01/01/" + match.group()


In [0]:
def add_january_one(source):
    """adds /01/01/ to year-only dates

    Args:
     source (pandas.Series): data with the dates

    Returns:
     pandas.Series: years in source with /01/01/ added
    """
    return clean(source, YEAR_ONLY, add_month_date)

In [0]:
two_digit_expression = GROUP.format(ONE_OR_TWO_DIGITS) + POSITIVE_LOOKAHEAD.format(SLASH)

def two_digits(match):
    """add a leading zero if needed

    Args:
     match (re.Match): match with one or two digits

    Returns:
     str: the matched string with leading zero if needed
    """
    # for some reason the string-formatting raises an error if it's a string
    # so cast it to an int
    return "{:02}".format(int(match.group()))

In [0]:
def clean_two_digits(source, sample=5):
    """makes sure source has two-digits

    Args:
     source (pandas.Series): data with digit followed by slash
     sample (int): number of samples to show

    Returns:
     pandas.Series: source with digits coerced to two digits
    """
    return clean(source, two_digit_expression, two_digits, sample)

In [0]:
def clean_two_digits_isolated(source, sample=5):
    """cleans two digits that are standalone

    Args:
     source (pandas.Series): source of the data
     sample (int): number of samples to show

    Returns:
     pandas.Series: converted data
    """
    return clean(source, ONE_OR_TWO_DIGITS, two_digits, sample)

In [0]:
digits = ("{:02}".format(month) for month in range(1, 13))
MONTH_TO_DIGITS = dict(zip(MONTH_PREFIXES, digits))
SHORT_MONTHS_EXPRESSION = OR.join((GROUP.format(month) for month in MONTH_TO_DIGITS))
def month_to_digits(match):
    """converts short month to digits

    Args:
     match (re.Match): object with short-month

    Returns:
     str: month as two-digit number (e.g. Jan -> 01)
    """
    return MONTH_TO_DIGITS[match.group()]

In [0]:
def convert_short_month_to_digits(source, sample=5):
    """converts three-letter months to two-digits

    Args:
     source (pandas.Series): data with three-letter months
     sample (int): number of samples to show

    Returns:
     pandas.Series: source with short-months coverted to digits
    """
    return clean(source,
                 SHORT_MONTHS_EXPRESSION,
                 month_to_digits,
                 sample)

In [0]:
def clean_months(source, sample=5):
    """clean up months (which start as words)

    Args:
     source (pandas.Series): source of the months
     sample (int): number of random samples to show
    """
    cleaned = clean_punctuation(source)

    print("Converting long months to short")
    cleaned = clean(cleaned,
                    LONG_TO_SHORT_EXPRESSION,
                    long_month_to_short, sample)

    print("Converting short months to digits")
    cleaned = clean(cleaned,
                    SHORT_MONTHS_EXPRESSION,
                    month_to_digits, sample)
    return cleaned

In [0]:
def frame_to_series(frame, index_source, samples=5):
    """re-combines data-frame into a series

    Args:
     frame (pandas.DataFrame): frame with month, day, year columns
     index_source (pandas.series): source to copy index from
     samples (index): number of random entries to print when done

    Returns:
     pandas.Series: series with dates as month/day/year
    """
    combined = frame.month + SLASH + frame.day + SLASH + frame.year
    combined.index = index_source.index
    print(combined.sample(samples))
    return combined

In [43]:
year_only_cleaned = add_january_one(year_only)

Random Sample Before:
     match
493  0        1978
473  0        1975
478  0        1993
496  0        2006
497  0        2008
Name: year_only, dtype: object

Random Sample After:
     match
470  0        01/01/1983
496  0        01/01/2006
495  0        01/01/1979
480  0        01/01/2013
483  0        01/01/1995
Name: year_only, dtype: object

Count of cleaned: 15


In [0]:
# Random Sample Before:
#      match
# 472  0        2010
# 495  0        1979
# 497  0        2008
# 481  0        1974
# 486  0        1973
# Name: year_only, dtype: object

# Random Sample After:
#      match
# 495  0        01/01/1979
# 470  0        01/01/1983
# 462  0        01/01/1988
# 481  0        01/01/1974
# 480  0        01/01/2013
# Name: year_only, dtype: object

# Count of cleaned: 15

In [45]:
leftovers_cleaned = add_january_one(leftovers)

Random Sample Before:
     match
459  0        1998
490  0        2007
487  0        1992
491  0        2009
469  0        2003
Name: leftovers, dtype: object

Random Sample After:
     match
463  0        01/01/2014
477  0        01/01/1994
455  0        01/01/1984
485  0        01/01/1987
498  0        01/01/2005
Name: leftovers, dtype: object

Count of cleaned: 30


In [0]:
# Random Sample Before:
#      match
# 487  0        1992
# 477  0        1994
# 498  0        2005
# 488  0        1977
# 484  0        2004
# Name: leftovers, dtype: object

# Random Sample After:
#      match
# 464  0        01/01/2016
# 455  0        01/01/1984
# 465  0        01/01/1976
# 475  0        01/01/2015
# 498  0        01/01/2005
# Name: leftovers, dtype: object

# Count of cleaned: 30

In [47]:
cleaned = pandas.concat([year_only_cleaned, leftovers_cleaned])
print(len(cleaned))

45


In [48]:
no_day_numeric_cleaned = clean_two_digits(no_day_numeric)

Random Sample Before:
     match
369  0        1/1978
425  0        7/2004
448  0        5/2010
362  0        8/2003
358  0        1/1983
Name: no_day_numeric, dtype: object

Random Sample After:
     match
423  0        12/1986
422  0        04/1973
448  0        05/2010
375  0        06/1973
410  0        01/2007
Name: no_day_numeric, dtype: object

Count of cleaned: 112


In [49]:
no_day_numeric_cleaned = clean(no_day_numeric_cleaned,
                               SLASH,
                               lambda m: "/01/")

Random Sample Before:
     match
377  0        06/2001
452  0        03/2003
360  0        12/2008
417  0        08/2000
439  0        10/2014
Name: no_day_numeric, dtype: object

Random Sample After:
     match
445  0        01/01/2008
381  0        01/01/2014
348  0        12/01/2005
425  0        07/01/2004
400  0        11/01/2008
Name: no_day_numeric, dtype: object

Count of cleaned: 112


In [0]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, no_day_numeric_cleaned])
assert len(cleaned) == no_day_numeric_count + original

In [51]:
print(len(cleaned))

157


In [52]:
no_day_cleaned = clean_months(no_day)

Cleaning Punctuation
Random Sample Before:
     match
304  0         Mar 2002
268  0         Dec 2009
337  0         Dec 2007
247  0         May 1983
338  0        Apr, 1998
Name: no_day, dtype: object

Random Sample After:
     match
257  0             Sep 2015
322  0         October 1991
308  0             Feb 1994
281  0             Aug 2004
311  0        February 1995
Name: no_day, dtype: object

Count of cleaned: 115
Converting long months to short
Random Sample Before:
     match
291  0            Jan 2004
241  0            May 2004
289  0            Sep 2007
238  0        January 1995
308  0            Feb 1994
Name: no_day, dtype: object

Random Sample After:
     match
293  0        Sep 2008
240  0        May 2011
268  0        Dec 2009
332  0        Jun 1974
255  0        Oct 2014
Name: no_day, dtype: object

Count of cleaned: 115
Converting short months to digits
Random Sample Before:
     match
232  0        Jul 1977
251  0        Dec 1998
228  0        Sep 1985
257  0     

In [53]:
no_day_cleaned = clean(no_day_cleaned,
                       SPACE + ONE_OR_MORE,
                       lambda match: "/01/")

Random Sample Before:
     match
277  0        04 2007
293  0        09 2008
312  0        02 1989
247  0        05 1983
246  0        07 1981
Name: no_day, dtype: object

Random Sample After:
     match
337  0        12/01/2007
306  0        05/01/2004
326  0        10/01/1995
278  0        09/01/1974
283  0        02/01/1977
Name: no_day, dtype: object

Count of cleaned: 115


In [54]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, no_day_cleaned])
print(len(cleaned))

272


In [0]:
assert len(cleaned) == no_day_count + original

In [56]:
frame = pandas.DataFrame(backwards.str.split().tolist(),
                         columns="day month year".split())
frame.head()

,day,month,year
0,24,Jan,2001
1,10,Sep,2004
2,26,May,1982
3,28,June,2002
4,06,May,1972


In [57]:
frame.day = clean_two_digits(frame.day)

Random Sample Before:
53    15
32    10
36    19
22    30
64    21
Name: day, dtype: object

Random Sample After:
47    23
60    17
10    10
43    13
34    26
Name: day, dtype: object

Count of cleaned: 69


In [58]:
frame.month = clean_months(frame.month)

Cleaning Punctuation
Converting long months to short
Random Sample Before:
20    Oct
58    May
42    Oct
31    May
67    May
Name: month, dtype: object

Random Sample After:
28    Jan
4     May
30    Oct
26    Jun
14    Oct
Name: month, dtype: object

Count of cleaned: 69
Converting short months to digits
Random Sample Before:
68    Jan
39    Oct
9     Jan
29    Mar
11    Feb
Name: month, dtype: object

Random Sample After:
21    11
6     10
4     05
55    12
33    08
Name: month, dtype: object

Count of cleaned: 69


In [59]:
backwards_cleaned = frame_to_series(frame, backwards)

     match
141  0        05/30/2016
146  0        11/11/2004
127  0        05/26/1982
164  0        10/21/1978
128  0        06/28/2002
dtype: object


In [0]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, backwards_cleaned])
assert len(cleaned) == original + backwards_count

In [61]:
print(len(cleaned))

341


In [62]:
frame = pandas.DataFrame(words.str.split().tolist(), columns="month day year".split())
print(frame.head())

      month  day  year
0     April  11,  1990
1       May  30,  2001
2       Feb  18,  1994
3  February  18,  1981
4  October.  11,  2013


In [63]:
frame.month = clean_months(frame.month)

Cleaning Punctuation
Random Sample Before:
28       May
31      June
11       Jan
5        Jan
13    August
Name: month, dtype: object

Random Sample After:
12    October
6        July
13     August
5         Jan
1         May
Name: month, dtype: object

Count of cleaned: 34
Converting long months to short
Random Sample Before:
32     January
16      August
3     February
25         Dec
22         Nov
Name: month, dtype: object

Random Sample After:
8     May
22    Nov
17    Apr
12    Oct
0     Apr
Name: month, dtype: object

Count of cleaned: 34
Converting short months to digits
Random Sample Before:
1     May
6     Jul
13    Aug
19    Jul
5     Jan
Name: month, dtype: object

Random Sample After:
7     12
13    08
6     07
4     10
15    07
Name: month, dtype: object

Count of cleaned: 34


In [64]:
frame.day = clean_punctuation(frame.day)

Cleaning Punctuation
Random Sample Before:
32    07,
6     26,
31    15,
5      24
14    01,
Name: day, dtype: object

Random Sample After:
6     26
2     18
30    25
1     30
15    25
Name: day, dtype: object

Count of cleaned: 34


In [65]:
frame.head()

,month,day,year
0,04,11,1990
1,05,30,2001
2,02,18,1994
3,02,18,1981
4,10,11,2013


In [66]:
words_cleaned = frame_to_series(frame, words)

     match
225  0        06/15/1972
219  0        12/14/1975
217  0        06/13/2011
208  0        09/01/2012
227  0        09/15/2011
dtype: object


In [67]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, words_cleaned])
assert len(cleaned) == original + words_count
print(len(cleaned))

375


In [68]:
print(twentieth.iloc[21])
twentieth_cleaned = twentieth.str.replace(DASH, SLASH)
print(cleaned.iloc[21])

4-13-82
01/01/1991


In [69]:
frame = pandas.DataFrame(twentieth_cleaned.str.split(SLASH).tolist(),
                         columns=["month", "day", "year"])
print(frame.head())

  month day year
0    03  25   93
1     6  18   85
2     7   8   71
3     9  27   75
4     2   6   96


In [70]:
frame.month = clean_two_digits_isolated(frame.month)

Random Sample Before:
16     5
22     9
41    11
76     3
58     7
Name: month, dtype: object

Random Sample After:
34    06
97    11
61    09
29    03
53    10
Name: month, dtype: object

Count of cleaned: 100


In [71]:
frame.day = clean_two_digits_isolated(frame.day)

Random Sample Before:
22    22
29    15
46    14
10    11
43    13
Name: day, dtype: object

Random Sample After:
65    29
8     07
32    14
81    22
12    01
Name: day, dtype: object

Count of cleaned: 100


In [72]:
frame.head()

,month,day,year
0,03,25,93
1,06,18,85
2,07,08,71
3,09,27,75
4,02,06,96


In [73]:
frame.year = clean(frame.year, TWO_DIGITS, lambda match: "19" + match.group())

Random Sample Before:
46    95
18    79
77    72
17    98
21    82
Name: year, dtype: object

Random Sample After:
94    1992
42    1991
77    1972
22    1989
16    1977
Name: year, dtype: object

Count of cleaned: 100


In [74]:
twentieth_cleaned = frame_to_series(frame, twentieth)

     match
108  0        05/04/1974
1    0        06/18/1985
48   0        09/17/1984
115  0        04/05/1989
116  0        12/04/1987
dtype: object


In [0]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, twentieth_cleaned])

In [0]:
assert len(cleaned) == original + twentieth_count

In [77]:
print(numeric.head())

    match
14  0         5/24/1990
15  0         1/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
Name: numeric, dtype: object


In [78]:
has_dashes = numeric.str.contains(DASH)
print(numeric[has_dashes])

Series([], Name: numeric, dtype: object)


In [79]:
frame = pandas.DataFrame(numeric.str.split(SLASH).tolist(),
                         columns="month day year".split())
print(frame.head())

  month day  year
0     5  24  1990
1     1  25  2011
2    10  13  1976
3    07  25  1984
4    03  31  1985


In [80]:
frame.month = clean_two_digits_isolated(frame.month)

Random Sample Before:
24     4
19    12
23     7
2     10
9     10
Name: month, dtype: object

Random Sample After:
2     10
17    08
18    04
10    12
19    12
Name: month, dtype: object

Count of cleaned: 25


In [81]:
frame.day = clean_two_digits_isolated(frame.day)

Random Sample Before:
22    11
4     31
12    29
11    20
13     3
Name: day, dtype: object

Random Sample After:
6    27
8    15
1    25
4    31
2    13
Name: day, dtype: object

Count of cleaned: 25


In [82]:
numeric_cleaned = frame_to_series(frame, numeric)

     match
45   0        02/15/1998
43   0        04/13/2002
15   0        01/25/2011
106  0        02/11/2006
83   0        09/14/2000
dtype: object


In [83]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, numeric_cleaned])
assert len(cleaned) == original + numeric_count
print(len(cleaned))

500


In [84]:
cleaned = pandas.concat([numeric_cleaned,
                         twentieth_cleaned,
                         words_cleaned,
                         backwards_cleaned,
                         no_day_cleaned,
                         no_day_numeric_cleaned,
                         year_only_cleaned,
                         leftovers_cleaned,
])
print(len(cleaned))
print(cleaned.head())
assert len(cleaned) == len(data)

500
    match
14  0        05/24/1990
15  0        01/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
dtype: object


In [85]:
print(cleaned.head())
datetimes = pandas.to_datetime(cleaned, format="%m/%d/%Y")
print(datetimes.head())

    match
14  0        05/24/1990
15  0        01/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
dtype: object
    match
14  0       1990-05-24
15  0       2011-01-25
17  0       1976-10-13
24  0       1984-07-25
30  0       1985-03-31
dtype: datetime64[ns]


In [86]:
sorted_dates = datetimes.sort_values()
print(sorted_dates.head())

    match
9   0       1971-04-10
84  0       1971-05-18
2   0       1971-07-08
53  0       1971-07-11
28  0       1971-09-12
dtype: datetime64[ns]


In [87]:
print(sorted_dates.tail())

     match
231  0       2016-05-01
141  0       2016-05-30
186  0       2016-10-13
161  0       2016-10-19
413  0       2016-11-01
dtype: datetime64[ns]


In [88]:
answer = pandas.Series(sorted_dates.index.labels[0])
print(answer.head())

0     9
1    84
2     2
3    53
4    28
dtype: int16


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  """Entry point for launching an IPython kernel.


In [0]:
def date_sorter():
    return answer